In [2]:
import os
import sys

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
#from tsdb import pickle_dump
import argparse
from unified_dataloader import UnifiedDataLoader
from saitsd import SAITS
sys.path.append("..")

from data_processing_utils import (
    window_truncate,
    random_mask,
    add_artificial_mask,
    saving_into_h5,
)
from datetime import datetime
import torch.nn as nn
import torch

TypeError: log(): argument 'input' (position 1) must be Tensor, not float

In [2]:
from scipy.stats import norm

In [3]:
mu = torch.tensor([1.0,2.0],requires_grad=True)
sigma = torch.tensor([1.0,2.0],requires_grad=True)
target= torch.tensor([1.0,2.0])

In [6]:
MODEL_DICT = {
    # Self-Attention (SA) based
   
    "SAITS": SAITS,
    # RNN based
    
}

In [3]:




def ornstein_uhlenbeck(x0, theta, mu, sigma, dt, num_steps, num_samples):
    x = np.zeros((num_steps, num_samples))
    x[0] = x0
    for t in range(1, num_steps):
        if t<7:
            x[t] = x[t-1] + theta * (mu - x[t-1]) * dt + sigma * np.sqrt(dt) * np.ones(num_samples)
        else:
            x[t] = x[t-1] + theta * (mu - x[t-1]) * dt + sigma * np.sqrt(dt) * np.random.normal(num_samples)
    return x

# Parameters
x0 = 0  # Initial value
theta = 0.1  # Rate of reversion
mu = 0.5  # Long-term mean
sigma = 0.3  # Volatility
dt = 0.01  # Time step
num_steps = 8  # Number of steps
num_samples =45 # Number of samples at each step

# Simulate Ornstein-Uhlenbeck process
ou_process = ornstein_uhlenbeck(x0, theta, mu, sigma, dt, num_steps, num_samples)




In [4]:
 # volatility
delta_t = 1  # time step size



dt = delta_t
mean = ou_process[6, :] * np.exp(-theta * dt) + mu * (1 - np.exp(-theta * dt))
variance = (sigma**2 / (2 * theta)) * (1 - np.exp(-2 * theta * dt))
ou_process[7, :] = mean + np.random.normal(0, np.sqrt(variance), num_samples)

In [5]:
empty=np.empty(0)

In [6]:
num_samples

45

In [7]:
for i in range(45):
    mean=np.repeat(ou_process[7, i] * np.exp(-theta * dt) + mu * (1 - np.exp(-theta * dt)),45)
    variance = np.repeat((sigma**2 / (2 * theta)) * (1 - np.exp(-2 * theta * dt)),45)
   
    ou_new=mean + np.random.normal(0, np.sqrt(variance),num_samples)
    empty=np.append(empty,ou_new)
    

In [8]:
empty

array([-0.32801771,  0.39889443, -0.24801868, ...,  0.27618894,
        0.16236607,  0.66707857])

In [9]:
df=pd.DataFrame(ou_process)

In [10]:
result_df = pd.DataFrame()


In [11]:
for col in df.columns:
    # Duplicate the current column 50 times and add to the result DataFrame
    duplicated_columns = pd.concat([df[col]] * 45, axis=1)
    # Rename the columns to indicate their original name and index
    duplicated_columns.columns = [f'{col}_{i+1}' for i in range(45)]
    # Append to the result DataFrame
    result_df = pd.concat([result_df, duplicated_columns], axis=1)



In [12]:
result_df = result_df.append(pd.Series(empty, index=result_df.columns), ignore_index=True)




In [13]:
result_df

,0_1,0_2,0_3,0_4,0_5,0_6,0_7,0_8,0_9,0_10,...,44_36,44_37,44_38,44_39,44_40,44_41,44_42,44_43,44_44,44_45
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.030500,0.030500,0.030500,0.030500,0.030500,0.030500,0.030500,0.030500,0.030500,0.030500,...,0.030500,0.030500,0.030500,0.030500,0.030500,0.030500,0.030500,0.030500,0.030500,0.030500
2,0.060969,0.060969,0.060969,0.060969,0.060969,0.060969,0.060969,0.060969,0.060969,0.060969,...,0.060969,0.060969,0.060969,0.060969,0.060969,0.060969,0.060969,0.060969,0.060969,0.060969
3,0.091409,0.091409,0.091409,0.091409,0.091409,0.091409,0.091409,0.091409,0.091409,0.091409,...,0.091409,0.091409,0.091409,0.091409,0.091409,0.091409,0.091409,0.091409,0.091409,0.091409
4,0.121817,0.121817,0.121817,0.121817,0.121817,0.121817,0.121817,0.121817,0.121817,0.121817,...,0.121817,0.121817,0.121817,0.121817,0.121817,0.121817,0.121817,0.121817,0.121817,0.121817
5,0.152195,0.152195,0.152195,0.152195,0.152195,0.152195,0.152195,0.152195,0.152195,0.152195,...,0.152195,0.152195,0.152195,0.152195,0.152195,0.152195,0.152195,0.152195,0.152195,0.152195
6,0.182543,0.182543,0.182543,0.182543,0.182543,0.182543,0.182543,0.182543,0.182543,0.182543,...,0.182543,0.182543,0.182543,0.182543,0.182543,0.182543,0.182543,0.182543,0.182543,0.182543
7,0.099839,0.099839,0.099839,0.099839,0.099839,0.099839,0.099839,0.099839,0.099839,0.099839,...,0.146144,0.146144,0.146144,0.146144,0.146144,0.146144,0.146144,0.146144,0.146144,0.146144
8,-0.328018,0.398894,-0.248019,0.107327,-0.089095,-0.207869,0.307163,-0.381622,0.042165,0.071697,...,-0.153329,0.110759,0.507117,0.687423,0.159373,0.122078,0.334667,0.276189,0.162366,0.667079


In [36]:
from scipy.stats import norm

In [39]:
norm.fit(ou_process[11])

(0.3280566451599556, 0.3067730689596443)

In [14]:
data=result_df.T.to_numpy()

In [15]:
data

array([[ 0.        ,  0.0305    ,  0.0609695 , ...,  0.18254311,
         0.09983927, -0.32801771],
       [ 0.        ,  0.0305    ,  0.0609695 , ...,  0.18254311,
         0.09983927,  0.39889443],
       [ 0.        ,  0.0305    ,  0.0609695 , ...,  0.18254311,
         0.09983927, -0.24801868],
       ...,
       [ 0.        ,  0.0305    ,  0.0609695 , ...,  0.18254311,
         0.14614433,  0.27618894],
       [ 0.        ,  0.0305    ,  0.0609695 , ...,  0.18254311,
         0.14614433,  0.16236607],
       [ 0.        ,  0.0305    ,  0.0609695 , ...,  0.18254311,
         0.14614433,  0.66707857]])

In [16]:
data=data.reshape(2025,9,1)

In [17]:
train_set_X=data[:1700]
test_set_X=data[1700:1800]
val_set_X=data[1800:2025]

In [18]:
train_set_dict = add_artificial_mask(
        train_set_X, 0.1, "train"
    )
val_set_dict = add_artificial_mask(val_set_X, 0.1, "val")
test_set_dict = add_artificial_mask(
        test_set_X, 0.1, "test"
    )

In [20]:
processed_data = {
        "train": train_set_dict,
        "val":val_set_dict,
        "test": test_set_dict,
    }

In [1]:
saving_dir='/Users/ivan/Downloads'

In [2]:
saving_into_h5(saving_dir, processed_data, classification_dataset=False)

NameError: name 'saving_into_h5' is not defined

In [3]:
from configparser import ConfigParser, ExtendedInterpolation

In [7]:
cfg = ConfigParser(interpolation=ExtendedInterpolation())

In [8]:
def read_arguments(arg_parser, cfg_parser):
    # file path
    arg_parser.dataset_base_dir = cfg_parser.get("file_path", "dataset_base_dir")
    
    # dataset info
    arg_parser.seq_len = cfg_parser.getint("dataset", "seq_len")
    arg_parser.batch_size = cfg_parser.getint("dataset", "batch_size")
    arg_parser.num_workers = cfg_parser.getint("dataset", "num_workers")
    arg_parser.feature_num = cfg_parser.getint("dataset", "feature_num")
    arg_parser.dataset_name = cfg_parser.get("dataset", "dataset_name")
    arg_parser.dataset_path = os.path.join(
        arg_parser.dataset_base_dir, arg_parser.dataset_name
    )
    arg_parser.eval_every_n_steps = cfg_parser.getint("dataset", "eval_every_n_steps")
    # training settings
    arg_parser.MIT = cfg_parser.getboolean("training", "MIT")
    arg_parser.ORT = cfg_parser.getboolean("training", "ORT")
    arg_parser.lr = cfg_parser.getfloat("training", "lr")
    arg_parser.optimizer_type = cfg_parser.get("training", "optimizer_type")
    arg_parser.weight_decay = cfg_parser.getfloat("training", "weight_decay")
    arg_parser.device = cfg_parser.get("training", "device")
    arg_parser.epochs = cfg_parser.getint("training", "epochs")
    arg_parser.early_stop_patience = cfg_parser.getint(
        "training", "early_stop_patience"
    )
    
    arg_parser.max_norm = cfg_parser.getfloat("training", "max_norm")
    arg_parser.imputation_loss_weight = cfg_parser.getfloat(
        "training", "imputation_loss_weight"
    )
    arg_parser.reconstruction_loss_weight = cfg_parser.getfloat(
        "training", "reconstruction_loss_weight"
    )
    # model settings
    arg_parser.model_name = cfg_parser.get("model", "model_name")
    arg_parser.model_type = cfg_parser.get("model", "model_type")
    return arg_parser

In [9]:

parser = argparse.ArgumentParser()
parser.add_argument("--config_path", type=str, help="path of config file",default="/Users/ivan/Desktop/saits/AirQuality_SAITS_best.ini")

_StoreAction(option_strings=['--config_path'], dest='config_path', nargs=None, const=None, default='/Users/ivan/Desktop/saits/AirQuality_SAITS_best.ini', type=<class 'str'>, choices=None, help='path of config file', metavar=None)

In [10]:
args,unknown = parser.parse_known_args()

In [11]:
args

Namespace(config_path='/Users/ivan/Desktop/saits/AirQuality_SAITS_best.ini')

In [12]:
cfg.read(args.config_path)

['/Users/ivan/Desktop/saits/AirQuality_SAITS_best.ini']

In [13]:
args = read_arguments(args, cfg)

In [14]:
args

Namespace(config_path='/Users/ivan/Desktop/saits/AirQuality_SAITS_best.ini', dataset_base_dir='generated_datasets', seq_len=9, batch_size=128, num_workers=4, feature_num=1, dataset_name='AirQuality_seqlen24_01masked', dataset_path='generated_datasets/AirQuality_seqlen24_01masked', eval_every_n_steps=7, MIT=True, ORT=True, lr=0.0008821387950693266, optimizer_type='adam', weight_decay=0.0, device='cuda', epochs=10000, early_stop_patience=30, max_norm=0.0, imputation_loss_weight=1.0, reconstruction_loss_weight=1.0, model_name='AirQuality_SAITS_best', model_type='SAITS')

In [15]:
if args.model_type in ["Transformer", "SAITS"]:  # if SA-based model
        args.input_with_mask = cfg.getboolean("model", "input_with_mask")
        args.n_groups = cfg.getint("model", "n_groups")
        args.n_group_inner_layers = cfg.getint("model", "n_group_inner_layers")
        args.param_sharing_strategy = cfg.get("model", "param_sharing_strategy")
        assert args.param_sharing_strategy in [
            "inner_group",
            "between_group",
        ], 'only "inner_group"/"between_group" sharing'
        args.d_model = cfg.getint("model", "d_model")
        args.d_inner = cfg.getint("model", "d_inner")
        args.n_head = cfg.getint("model", "n_head")
        args.d_k = cfg.getint("model", "d_k")
        args.d_v = cfg.getint("model", "d_v")
        args.dropout = cfg.getfloat("model", "dropout")
        args.diagonal_attention_mask = cfg.getboolean(
            "model", "diagonal_attention_mask"
        )

In [16]:
dict_args = vars(args)

In [17]:
model_args = {
            "device":'cpu',
            "MIT": args.MIT,
            # imputer args
            "n_groups": dict_args["n_groups"],
            "n_group_inner_layers": args.n_group_inner_layers,
            "d_time": args.seq_len,
            "d_feature": args.feature_num,
            "dropout": dict_args["dropout"],
            "d_model": dict_args["d_model"],
            "d_inner": dict_args["d_inner"],
            "n_head": dict_args["n_head"],
            "d_k": dict_args["d_k"],
            "d_v": dict_args["d_v"],
            "input_with_mask": args.input_with_mask,
            "diagonal_attention_mask": args.diagonal_attention_mask,
            "param_sharing_strategy": args.param_sharing_strategy,
        }

In [18]:
time_now = datetime.now().__format__("%Y-%m-%d_T%H:%M:%S")

In [19]:
unified_dataloader = UnifiedDataLoader(
        saving_dir,
        args.seq_len,
        1,
        args.model_type,
        args.batch_size,
        args.num_workers,
        args.MIT,
    )

In [20]:
train_dataloader,val_dataloader= unified_dataloader.get_train_val_dataloader()

In [21]:
train_dataloader

In [22]:
model = MODEL_DICT[args.model_type](**model_args)
OPTIMIZER = {"adam": torch.optim.Adam, "adamw": torch.optim.AdamW}

In [23]:
indices, X, missing_mask, X_holdout, indicating_mask = map(
                    lambda x: x.to('cpu'),next(iter(train_dataloader))
                )

In [26]:
inputs = {
                    "indices": indices,
                    "X": X,
                    "missing_mask": missing_mask,
                    "X_holdout": X_holdout,
                    "indicating_mask": indicating_mask,
                }

In [27]:
def result_processing(results):
    """process results and losses for each training step"""
    results["total_loss"] = torch.tensor(0.0, device=args.device)
    if args.MIT:
        results["total_loss"] += results["imputation_loss"]
    return results

In [28]:
def process_each_training_step(
    results, optimizer
):
    """process each training step and return whether to early stop"""
  
    # apply gradient clipping if args.max_norm != 0
    if args.max_norm != 0:
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=args.max_norm)
    results["total_loss"].backward()
    optimizer.step()
    return False

In [34]:
def model_processing(
    data,
    model,
    stage,
    # following arguments are only required in the training stage
    optimizer=None,
    val_dataloader=None,
 
    training_controller=None,
    logger=None,
):
    if stage == "train":
        optimizer.zero_grad()
        if not args.MIT:
              # then for self-attention based models, i.e. Transformer/SAITS
                indices, X, missing_mask = map(lambda x: x.to(args.device), data)
                inputs = {"indices": indices, "X": X, "missing_mask": missing_mask}
                results = result_processing(model(inputs, stage))
                early_stopping = process_each_training_step(
                results,
                optimizer,
                val_dataloader,
                training_controller,

            )
        else:
            
                
                indices, X, missing_mask, X_holdout, indicating_mask = map(
                    lambda x: x.to('cpu'), data
                    )
                for idx in range(7,9):
                    inputs = {"indices": indices, "X": X, "missing_mask": missing_mask,"X_holdout":X_holdout,"indicating_mask":indicating_mask,"idx":idx}
                    
                    results = result_processing(model(inputs, stage))
                    X[:,idx,:]=X_holdout[:,idx,:]
                    missing_mask[:,idx,:]=1
                    early_stopping = process_each_training_step(
                    results,
                    optimizer
                    )
        return early_stopping
    return inputs, results

  


In [35]:
def train(
    model,
    optimizer,
    train_dataloader
):
    for epoch in range(args.epochs):
        early_stopping = False
        args.final_epoch = True if epoch == args.epochs - 1 else False
        for idx, data in enumerate(train_dataloader):
            model.train()
            early_stopping = model_processing(
                data,
                model,
                "train",
                optimizer
            )

In [36]:
args

Namespace(config_path='/Users/ivan/Desktop/saits/AirQuality_SAITS_best.ini', dataset_base_dir='generated_datasets', seq_len=9, batch_size=128, num_workers=4, feature_num=1, dataset_name='AirQuality_seqlen24_01masked', dataset_path='generated_datasets/AirQuality_seqlen24_01masked', eval_every_n_steps=7, MIT=True, ORT=True, lr=0.0008821387950693266, optimizer_type='adam', weight_decay=0.0, device='cuda', epochs=10000, early_stop_patience=30, max_norm=0.0, imputation_loss_weight=1.0, reconstruction_loss_weight=1.0, model_name='AirQuality_SAITS_best', model_type='SAITS', input_with_mask=True, n_groups=1, n_group_inner_layers=1, param_sharing_strategy='inner_group', d_model=32, d_inner=32, n_head=4, d_k=128, d_v=64, dropout=0.0, diagonal_attention_mask=True, final_epoch=False)

In [37]:
optimizer = OPTIMIZER[args.optimizer_type](
            model.parameters(), lr=dict_args["lr"], weight_decay=args.weight_decay
        )

In [38]:
train(
            model,
            optimizer,
            train_dataloader
        )



: 